# Introduction

The goal of this assignment is to create a basic program that provides an overview of basic evaluation metrics (in particular, precision, recall, f-score and a confusion matrix) from documents provided in the conll format. 
You will need to implement the calculations for precision, recall and f-score yourself (i.e. do not use an existing module that spits them out). Make sure that your code can handle the situation where there are no true positives for a specific class.

This notebook provides functions for reading in conll structures with pandas and proposes a structure for calculating your evaluation metrics and producing the confusion matrix. Feel free to adjust the proposed structure if you see fit.

In [62]:
import sys
import pandas as pd
# see tips & tricks on using defaultdict (remove when you do not use it)
from collections import defaultdict, Counter
import numpy as np
# module for verifying output
from nose.tools import assert_equal

# A note Pandas

Pandas is a module that provides data structures and is widely used for dealing with data representations in machine learning. It is a bit more advanced than the csv module we saw in the preprocessing notebook.
Working with pandas data structures can be tricky, but it will generally work well if you follow online tutorials and examples closely. If your code is slow before you even started training your models, it is likely to be a problem with the way you are using Pandas (it will still work in most cases, you will just have to wait a bit longer). Once you are more used to working with modules and complex objects, it will also become easier to work with Pandas.

In the examples below, we assume that the 

In [63]:
def extract_annotations(inputfile, annotationcolumn, delimiter='\t'):
    '''
    This function extracts annotations represented in the conll format from a file
    
    :param inputfile: the path to the conll file
    :param annotationcolumn: the name of the column in which the target annotation is provided
    :param delimiter: optional parameter to overwrite the default delimiter (tab)
    :type inputfile: string
    :type annotationcolumn: string
    :type delimiter: string
    :returns: the annotations as a list
    '''
    #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
    conll_input = pd.read_csv(inputfile, sep=delimiter)
    annotations = conll_input[annotationcolumn].tolist()
    return annotations

In [64]:
def obtain_counts(goldannotations, machineannotations):
    '''
    This function compares the gold annotations to machine output
    
    :param goldannotations: the gold annotations
    :param machineannotations: the output annotations of the system in question
    :type goldannotations: the type of the object created in extract_annotations
    :type machineannotations: the type of the object created in extract_annotations
    
    :returns: a countainer providing the counts for each predicted and gold class pair
    '''
    
    # TIP on how to get the counts for each class
    # https://stackoverflow.com/questions/49393683/how-to-count-items-in-a-nested-dictionary, last accessed 22.10.2020
    evaluation_counts = defaultdict(Counter)
    assert len(goldannotations) == len(machineannotations)
    for i in range(len(goldannotations)):
        evaluation_counts[goldannotations[i]][machineannotations[i]] += 1
    return evaluation_counts
    
def calculate_precision_recall_fscore(evaluation_counts):
    '''
    Calculate precision recall and fscore for each class and return them in a dictionary
    
    :param evaluation_counts: a container from which you can obtain the true positives, false positives and false negatives for each class
    :type evaluation_counts: type of object returned by obtain_counts
    
    :returns the precision, recall and f-score of each class in a container
    '''
    
    # TIP: you may want to write a separate function that provides an overview of true positives, false positives and false negatives
    #      for each class based on the outcome of obtain counts
    # YOUR CODE HERE (and remove statement below)
    value_dict = defaultdict(Counter)

    macro_list_prec = []
    macro_list_rec = []
    macro_list_fscore = []
    macro_count = 0
    
    for label in evaluation_counts.keys():
        # Calculation of TP, FP & FN
        TP = evaluation_counts[label][label]
        FP = 0
        FN = 0
        for label_ in evaluation_counts.keys():
            if label_ != label:
                FP += evaluation_counts[label_][label]
                FN += evaluation_counts[label][label_]

        # Precision, Recall & F-Score for class 'label'
        try:
            precis = TP / (TP + FP)
        except ZeroDivisionError:
            precis = 0
        try:
            recall = TP / (TP + FN)
        except ZeroDivisionError:
            recall = 0
        try:
            FScore = (2 * precis * recall) / (precis + recall)
        except ZeroDivisionError:
            FScore = 0
        
        # Assign Precision, Recall, & F-Score according to set structure
        value_dict[label]['precision'] = precis
        value_dict[label]['recall'] = recall
        value_dict[label]['f-score'] = FScore

        macro_list_fscore.append(FScore)
        macro_list_prec.append(precis)
        macro_list_rec.append(recall)
        macro_count += 1
    
    value_dict['macro']['precision'] = sum(macro_list_prec) / float(macro_count)
    value_dict['macro']['recall'] = sum(macro_list_rec) / float(macro_count)
    value_dict['macro']['f-score'] = sum(macro_list_fscore) / float(macro_count)

    return value_dict
            

def provide_confusion_matrix(evaluation_counts):
    '''
    Read in the evaluation counts and provide a confusion matrix for each class
    
    :param evaluation_counts: a container from which you can obtain the true positives, false positives and false negatives for each class
    :type evaluation_counts: type of object returned by obtain_counts
    
    :prints out a confusion matrix
    '''
    
    # TIP: provide_output_tables does something similar, but those tables are assuming one additional nested layer
    #      your solution can thus be a simpler version of the one provided in provide_output_tables below
    
    # YOUR CODE HERE (and remove statement below)
    res = pd.DataFrame.from_dict(evaluation_counts).fillna(0)

    print(res)
    print(res.to_latex())

In [65]:
def carry_out_evaluation(gold_annotations, systemfile, systemcolumn, delimiter='\t'):
    '''
    Carries out the evaluation process (from input file to calculating relevant scores)
    
    :param gold_annotations: list of gold annotations
    :param systemfile: path to file with system output
    :param systemcolumn: indication of column with relevant information
    :param delimiter: specification of formatting of file (default delimiter set to '\t')
    
    returns evaluation information for this specific system
    '''
    system_annotations = extract_annotations(systemfile, systemcolumn, delimiter)
    evaluation_counts = obtain_counts(gold_annotations, system_annotations)
    provide_confusion_matrix(evaluation_counts)
    evaluation_outcome = calculate_precision_recall_fscore(evaluation_counts)
    
    return evaluation_outcome

In [66]:
def provide_output_tables(evaluations):
    '''
    Create tables based on the evaluation of various systems
    
    :param evaluations: the outcome of evaluating one or more systems
    '''
    #https:stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
    evaluations_pddf = pd.DataFrame.from_dict({(i,j): evaluations[i][j]
                                              for i in evaluations.keys()
                                              for j in evaluations[i].keys()},
                                             orient='index')
    print(evaluations_pddf)
    print(evaluations_pddf.to_latex())

In [67]:
def run_evaluations(goldfile, goldcolumn, systems):
    '''
    Carry out standard evaluation for one or more system outputs
    
    :param goldfile: path to file with goldstandard
    :param goldcolumn: indicator of column in gold file where gold labels can be found
    :param systems: required information to find and process system output
    :type goldfile: string
    :type goldcolumn: integer
    :type systems: list (providing file name, information on tab with system output and system name for each element)
    
    :returns the evaluations for all systems
    '''
    evaluations = {}
    #not specifying delimiters here, since it corresponds to the default ('\t')
    gold_annotations = extract_annotations(goldfile, goldcolumn)
    for system in systems:
        sys_evaluation = carry_out_evaluation(gold_annotations, system[0], system[1])
        evaluations[system[2]] = sys_evaluation
    return evaluations

# Checking the overall set-up

The functions below illustrate how to run the setup as outlined above using a main function and, later, commandline arguments. This setup will facilitate the transformation to an experimental setup that no longer makes use of notebooks, that you will submit later on. There are also some functions that can be used to test your implementation You can carry out a few small tests yourself with the data provided in the data/ folder.

In [68]:
def identify_evaluation_value(system, class_label, value_name, evaluations):
    '''
    Return the outcome of a specific value of the evaluation
    
    :param system: the name of the system
    :param class_label: the name of the class for which the value should be returned
    :param value_name: the name of the score that is returned
    :param evaluations: the overview of evaluations
    
    :returns the requested value
    '''
    return evaluations[system][class_label][value_name]

In [69]:
def create_system_information(system_information):
    '''
    Takes system information in the form that it is passed on through sys.argv or via a settingsfile
    and returns a list of elements specifying all the needed information on each system output file to carry out the evaluation.
    
    :param system_information is the input as from a commandline or an input file
    '''
    # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    systems_list = [system_information[i:i + 3] for i in range(0, len(system_information), 3)]
    return systems_list

In [70]:

def main(my_args=None):
    
    if my_args is None:
        my_args = sys.argv
    
    system_info = create_system_information(my_args[2:])
    evaluations = run_evaluations(my_args[0], my_args[1], system_info)
    provide_output_tables(evaluations)
    check_eval = identify_evaluation_value('system1', 'O', 'f-score', evaluations)
    #if it does not work, this assert stateme
    assert_equal("%.3f" % check_eval,"0.889")
    

# these can come from the commandline using sys.argv for instance
my_args = ['../../data/archived_data/minigold.csv','gold','../../data/archived_data/miniout1.csv','NER','system1']
main(my_args)

         O  B-ORG  I-ORG  B-PER  I-PER  B-MISC
O      8.0    0.0    1.0    0.0    0.0     1.0
B-ORG  0.0    2.0    0.0    0.0    0.0     0.0
I-ORG  0.0    0.0    2.0    0.0    0.0     0.0
B-PER  0.0    0.0    0.0    1.0    0.0     0.0
I-PER  0.0    0.0    0.0    0.0    1.0     0.0
\begin{tabular}{lrrrrrr}
\toprule
{} &    O &  B-ORG &  I-ORG &  B-PER &  I-PER &  B-MISC \\
\midrule
O     &  8.0 &    0.0 &    1.0 &    0.0 &    0.0 &     1.0 \\
B-ORG &  0.0 &    2.0 &    0.0 &    0.0 &    0.0 &     0.0 \\
I-ORG &  0.0 &    0.0 &    2.0 &    0.0 &    0.0 &     0.0 \\
B-PER &  0.0 &    0.0 &    0.0 &    1.0 &    0.0 &     0.0 \\
I-PER &  0.0 &    0.0 &    0.0 &    0.0 &    1.0 &     0.0 \\
\bottomrule
\end{tabular}

                precision    recall   f-score
system1 O             0.8  1.000000  0.888889
        B-ORG         1.0  1.000000  1.000000
        I-ORG         1.0  0.666667  0.800000
        B-PER         1.0  1.000000  1.000000
        I-PER         1.0  1.000000  1.000000
   

In [71]:
#some additional tests

test_args = ['../../data/archived_data/minigold.csv','gold','../../data/archived_data/miniout2.csv','NER','system2']
system_info = create_system_information(test_args[2:])
evaluations = run_evaluations(test_args[0], test_args[1], system_info)
test_eval = identify_evaluation_value('system2', 'I-ORG', 'f-score', evaluations)
assert_equal("%.3f" % test_eval,"0.571")
test_eval2 = identify_evaluation_value('system2', 'I-PER', 'precision', evaluations)
assert_equal("%.3f" % test_eval2,"0.500")
test_eval3 = identify_evaluation_value('system2', 'I-ORG', 'recall', evaluations)
assert_equal("%.3f" % test_eval3,"0.667")

         O  B-ORG  I-ORG  B-PER  I-PER  B-MISC
O      8.0    0.0    1.0    0.0    0.0     1.0
I-ORG  0.0    2.0    2.0    0.0    0.0     0.0
I-PER  0.0    0.0    0.0    1.0    1.0     0.0
\begin{tabular}{lrrrrrr}
\toprule
{} &    O &  B-ORG &  I-ORG &  B-PER &  I-PER &  B-MISC \\
\midrule
O     &  8.0 &    0.0 &    1.0 &    0.0 &    0.0 &     1.0 \\
I-ORG &  0.0 &    2.0 &    2.0 &    0.0 &    0.0 &     0.0 \\
I-PER &  0.0 &    0.0 &    0.0 &    1.0 &    1.0 &     0.0 \\
\bottomrule
\end{tabular}



In [72]:
def compare_outcome(label_set_1, label_set_2):
    eval_counts = obtain_counts(label_set_1, label_set_2)

    values = calculate_precision_recall_fscore(eval_counts)
    values = pd.DataFrame.from_dict(values).fillna(0)
    print(values.to_latex())

    eval_counts = pd.DataFrame.from_dict(eval_counts).fillna(0)
    print(eval_counts.to_latex())

# Assignment 1 Evaluation

In [73]:
# comparing gold w/ stanford & spacy

# Loading data and extracting features
gold = pd.read_csv('../../data/gold_stripped.conll', sep='\t', header=None)
spacy = pd.read_csv('../../data/spacy_out_matched_tokens-preprocessed.conll', sep='\t', header=None)
stanf = pd.read_csv('../../data/stanford_out_matched_tokens-preprocessed.conll', sep='\t', header=None)
gold_prep = pd.read_csv('../../data/gold_stripped-preprocessed.conll', sep='\t', header=None)

gold_out_actual = gold[7]
spacy_out = spacy[2].replace(np.nan, 'O')
stanf_out = stanf[3]
gold_prep_out = gold_prep[3]

In [74]:
# Gold vs. Spacy
compare_outcome(gold_prep_out, spacy_out)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &       ORG &       LOC &      MISC &       PER &     macro \\
\midrule
precision &  0.968362 &  0.471852 &  0.764790 &  0.693193 &  0.830780 &  0.745795 \\
recall    &  0.975856 &  0.524857 &  0.765521 &  0.570189 &  0.741494 &  0.715583 \\
f-score   &  0.972095 &  0.496945 &  0.765155 &  0.625703 &  0.783602 &  0.728700 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &      O &   ORG &   LOC &  MISC &   PER \\
\midrule
PER  &    146 &   196 &    80 &    53 &  2332 \\
O    &  41106 &   502 &   204 &   278 &   359 \\
ORG  &    620 &  1098 &   134 &   154 &   321 \\
MISC &    140 &    73 &    73 &   723 &    34 \\
LOC  &    111 &   223 &  1603 &    60 &    99 \\
\bottomrule
\end{tabular}



In [75]:
# Gold vs. Stanford
compare_outcome(gold_prep_out, stanf_out)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &       ORG &       LOC &      MISC &       PER &     macro \\
\midrule
precision &  0.988773 &  0.906392 &  0.756153 &  0.938776 &  0.971456 &  0.912310 \\
recall    &  0.997294 &  0.860899 &  0.953677 &  0.362776 &  0.963116 &  0.827552 \\
f-score   &  0.993015 &  0.883060 &  0.843506 &  0.523322 &  0.967268 &  0.842034 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &      O &   ORG &   LOC &  MISC &   PER \\
\midrule
O    &  42009 &   149 &    50 &   211 &    67 \\
LOC  &     22 &    84 &  1997 &   518 &    20 \\
ORG  &     67 &  1801 &    34 &    60 &    25 \\
PER  &      9 &    54 &     7 &    19 &  3029 \\
MISC &     16 &     4 &     6 &   460 &     4 \\
\bottomrule
\end{tabular}



# Assignment 2 Evaluation

## Basic Features (No added features, same as Assignment1)

In [76]:
# comparing gold w/ stanford & spacy

# Loading data and extracting features
logreg = pd.read_csv('../../data/out.logreg.conll', sep='\t', header=None)
NB = pd.read_csv('../../data/out.NB.conll', sep='\t', header=None)
SVM = pd.read_csv('../../data/out.SVM.conll', sep='\t', header=None)

logreg_out = logreg[4]
logreg_out_actual = logreg[3]
nb_out = NB[4]
nb_out_actual = NB[3]
svm_out = SVM[4]
svm_out_actual = SVM[3]

FileNotFoundError: [Errno 2] File ../../data/out.logreg.conll does not exist: '../../data/out.logreg.conll'

In [101]:
compare_outcome(logreg_out, logreg_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-LOC &    I-MISC &     I-PER &     I-ORG &     macro \\
\midrule
precision &  0.998766 &  0.717765 &  0.625791 &  0.547854 &  0.521989 &  0.682433 \\
recall    &  0.934766 &  0.866282 &  0.901938 &  0.959354 &  0.838066 &  0.900081 \\
f-score   &  0.965706 &  0.785061 &  0.738907 &  0.697430 &  0.643299 &  0.766081 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &      O &   I-LOC &  I-MISC &   I-PER &   I-ORG \\
\midrule
O      &  42071 &     4.0 &      20 &     3.0 &    25.0 \\
I-ORG  &    764 &   156.0 &      41 &    39.0 &  1092.0 \\
I-LOC  &    412 &  1503.0 &      20 &    16.0 &   143.0 \\
I-PER  &   1381 &    21.0 &       5 &  1723.0 &    15.0 \\
I-MISC &    379 &    51.0 &     791 &    15.0 &    28.0 \\
B-MISC &      1 &     0.0 &       3 &     0.0 &     0.0 \\
\bottomrule
\end{tabular}



In [102]:
compare_outcome(nb_out, nb_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-LOC &    I-MISC &     I-PER &     I-ORG &     macro \\
\midrule
precision &  0.999193 &  0.717765 &  0.571203 &  0.547218 &  0.583652 &  0.683806 \\
recall    &  0.936456 &  0.870295 &  0.931613 &  0.959309 &  0.826676 &  0.904870 \\
f-score   &  0.966808 &  0.786705 &  0.708190 &  0.696902 &  0.684225 &  0.768566 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &      O &   I-LOC &  I-MISC &   I-PER &   I-ORG \\
\midrule
O      &  42089 &     3.0 &      11 &     3.0 &    17.0 \\
I-ORG  &    650 &   150.0 &      32 &    39.0 &  1221.0 \\
I-LOC  &    411 &  1503.0 &       8 &    16.0 &   156.0 \\
I-PER  &   1383 &    20.0 &       2 &  1721.0 &    19.0 \\
I-MISC &    412 &    51.0 &     722 &    15.0 &    64.0 \\
B-MISC &      2 &     0.0 &       2 &     0.0 &     0.0 \\
\bottomrule
\end{tabular}



In [103]:
compare_outcome(svm_out, svm_out_actual)

\begin{tabular}{lrrrrrrr}
\toprule
{} &         O &     I-LOC &    I-MISC &     I-PER &     I-ORG &  B-MISC &     macro \\
\midrule
precision &  0.998077 &  0.738777 &  0.681962 &  0.646423 &  0.644359 &     0.0 &  0.618266 \\
recall    &  0.950510 &  0.865213 &  0.898853 &  0.955357 &  0.834675 &     0.0 &  0.750768 \\
f-score   &  0.973713 &  0.797012 &  0.775529 &  0.771098 &  0.727273 &     0.0 &  0.674104 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrr}
\toprule
{} &      O &   I-LOC &  I-MISC &   I-PER &   I-ORG &  B-MISC \\
\midrule
O      &  42042 &     5.0 &      26 &     8.0 &    42.0 &     0.0 \\
I-ORG  &    491 &   162.0 &      42 &    49.0 &  1348.0 &     0.0 \\
I-PER  &   1064 &    23.0 &       5 &  2033.0 &    20.0 &     0.0 \\
I-MISC &    285 &    51.0 &     862 &    17.0 &    48.0 &     1.0 \\
I-LOC  &    348 &  1547.0 &      21 &    21.0 &   157.0 &     0.0 \\
B-MISC &      1 &     0.0 &       3 &     0.0 &     0.0 &     0.0 \\
\bottomrule
\end{tabular}



## With Added Features

In [104]:
# comparing gold w/ stanford & spacy

# Loading data and extracting features
logreg = pd.read_csv('../../data/out_added_feats.logreg.conll', sep='\t', header=None)
NB = pd.read_csv('../../data/out_added_feats.NB.conll', sep='\t', header=None)
SVM = pd.read_csv('../../data/out_added_feats.SVM.conll', sep='\t', header=None)

logreg_out = logreg[4]
logreg_out_actual = logreg[3]
nb_out = NB[4]
nb_out_actual = NB[3]
svm_out = SVM[4]
svm_out_actual = SVM[3]

In [105]:
compare_outcome(logreg_out, logreg_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-ORG &     I-LOC &     I-PER &    I-MISC &     macro \\
\midrule
precision &  0.993733 &  0.720363 &  0.831423 &  0.920191 &  0.682753 &  0.829693 \\
recall    &  0.987031 &  0.760727 &  0.860178 &  0.861310 &  0.914195 &  0.876688 \\
f-score   &  0.990371 &  0.739995 &  0.845556 &  0.889777 &  0.781703 &  0.849480 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &        O &   I-ORG &   I-LOC &  I-PER &  I-MISC \\
\midrule
O      &  41859.0 &   159.0 &    16.0 &     64 &      25 \\
I-PER  &    126.0 &    68.0 &    54.0 &   2894 &       3 \\
I-ORG  &    183.0 &  1507.0 &   161.0 &    212 &      29 \\
I-MISC &    172.0 &   107.0 &    52.0 &     70 &     863 \\
I-LOC  &     69.0 &   140.0 &  1741.0 &    120 &      24 \\
B-MISC &      0.0 &     0.0 &     0.0 &      1 &       3 \\
\bottomrule
\end{tabular}



In [106]:
compare_outcome(nb_out, nb_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &     I-PER &         O &     I-ORG &     I-LOC &    I-MISC &     macro \\
\midrule
precision &  0.917647 &  0.985091 &  0.708413 &  0.820439 &  0.564873 &  0.799293 \\
recall    &  0.791552 &  0.985512 &  0.709430 &  0.808090 &  0.949468 &  0.848811 \\
f-score   &  0.849948 &  0.985302 &  0.708921 &  0.814218 &  0.708333 &  0.813345 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &  I-PER &        O &  I-ORG &   I-LOC &  I-MISC \\
\midrule
O      &    304 &  41495.0 &    217 &    93.0 &      14 \\
I-ORG  &    199 &    228.0 &   1482 &   171.0 &      12 \\
I-PER  &   2886 &    116.0 &     72 &    67.0 &       4 \\
I-LOC  &    137 &     96.0 &    135 &  1718.0 &       8 \\
I-MISC &    120 &    170.0 &    183 &    77.0 &     714 \\
B-MISC &      1 &      0.0 &      1 &     0.0 &       2 \\
\bottomrule
\end{tabular}



In [107]:
compare_outcome(svm_out, svm_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-ORG &     I-LOC &     I-PER &    I-MISC &     macro \\
\midrule
precision &  0.993851 &  0.716061 &  0.819007 &  0.925278 &  0.687500 &  0.828339 \\
recall    &  0.991380 &  0.758481 &  0.868794 &  0.818796 &  0.880446 &  0.863579 \\
f-score   &  0.992614 &  0.736661 &  0.843166 &  0.868786 &  0.772101 &  0.842666 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &        O &   I-ORG &   I-LOC &  I-PER &  I-MISC \\
\midrule
O      &  41864.0 &   110.0 &    18.0 &     85 &      46 \\
I-PER  &     94.0 &    72.0 &    50.0 &   2910 &      19 \\
I-ORG  &    118.0 &  1498.0 &   145.0 &    303 &      28 \\
I-MISC &    101.0 &   142.0 &    46.0 &    106 &     869 \\
I-LOC  &     51.0 &   153.0 &  1715.0 &    150 &      25 \\
B-MISC &      0.0 &     0.0 &     0.0 &      1 &       3 \\
\bottomrule
\end{tabular}



## With Embeddings

In [108]:
# comparing gold w/ stanford & spacy

# Loading data and extracting features
logreg = pd.read_csv('../../data/out_word2vec.logreg.conll', sep='\t', header=None)
NB = pd.read_csv('../../data/out_word2vec.NB.conll', sep='\t', header=None)
SVM = pd.read_csv('../../data/out_word2vec.SVM.conll', sep='\t', header=None)

logreg_out = logreg[4]
logreg_out_actual = logreg[3]
nb_out = NB[4]
nb_out_actual = NB[3]
svm_out = SVM[4]
svm_out_actual = SVM[3]

In [109]:
compare_outcome(nb_out, nb_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-LOC &    I-MISC &     I-PER &     I-ORG &     macro \\
\midrule
precision &  0.999193 &  0.717765 &  0.571203 &  0.547218 &  0.583652 &  0.683806 \\
recall    &  0.936456 &  0.870295 &  0.931613 &  0.959309 &  0.826676 &  0.904870 \\
f-score   &  0.966808 &  0.786705 &  0.708190 &  0.696902 &  0.684225 &  0.768566 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &      O &   I-LOC &  I-MISC &   I-PER &   I-ORG \\
\midrule
O      &  42089 &     3.0 &      11 &     3.0 &    17.0 \\
I-ORG  &    650 &   150.0 &      32 &    39.0 &  1221.0 \\
I-LOC  &    411 &  1503.0 &       8 &    16.0 &   156.0 \\
I-PER  &   1383 &    20.0 &       2 &  1721.0 &    19.0 \\
I-MISC &    412 &    51.0 &     722 &    15.0 &    64.0 \\
B-MISC &      2 &     0.0 &       2 &     0.0 &     0.0 \\
\bottomrule
\end{tabular}



In [110]:
compare_outcome(logreg_out, logreg_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-LOC &    I-MISC &     I-PER &     I-ORG &     macro \\
\midrule
precision &  0.998766 &  0.717765 &  0.625791 &  0.547854 &  0.521989 &  0.682433 \\
recall    &  0.934766 &  0.866282 &  0.901938 &  0.959354 &  0.838066 &  0.900081 \\
f-score   &  0.965706 &  0.785061 &  0.738907 &  0.697430 &  0.643299 &  0.766081 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &      O &   I-LOC &  I-MISC &   I-PER &   I-ORG \\
\midrule
O      &  42071 &     4.0 &      20 &     3.0 &    25.0 \\
I-ORG  &    764 &   156.0 &      41 &    39.0 &  1092.0 \\
I-LOC  &    412 &  1503.0 &      20 &    16.0 &   143.0 \\
I-PER  &   1381 &    21.0 &       5 &  1723.0 &    15.0 \\
I-MISC &    379 &    51.0 &     791 &    15.0 &    28.0 \\
B-MISC &      1 &     0.0 &       3 &     0.0 &     0.0 \\
\bottomrule
\end{tabular}



In [111]:
compare_outcome(svm_out, svm_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-ORG &     I-LOC &    I-MISC &     I-PER &     macro \\
\midrule
precision &  0.998457 &  0.811185 &  0.903534 &  0.805380 &  0.917965 &  0.887304 \\
recall    &  0.985011 &  0.916802 &  0.905263 &  0.926297 &  0.968792 &  0.940433 \\
f-score   &  0.991688 &  0.860766 &  0.904398 &  0.861617 &  0.942694 &  0.912232 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &        O &   I-ORG &   I-LOC &  I-MISC &   I-PER \\
\midrule
O      &  42058.0 &    23.0 &     4.0 &      29 &     9.0 \\
I-MISC &    140.0 &    26.0 &    60.0 &    1018 &    20.0 \\
I-LOC  &     66.0 &    97.0 &  1892.0 &      10 &    29.0 \\
I-PER  &    245.0 &     8.0 &     2.0 &       3 &  2887.0 \\
I-ORG  &    189.0 &  1697.0 &   132.0 &      39 &    35.0 \\
B-MISC &      0.0 &     0.0 &     0.0 &       4 &     0.0 \\
\bottomrule
\end{tabular}



## Word embeddings + added features

In [112]:
# comparing gold w/ stanford & spacy

# Loading data and extracting features
logreg = pd.read_csv('../../data/out_word2vec_added_feats.logreg.conll', sep='\t', header=None)
NB = pd.read_csv('../../data/out_word2vec_added_feats.NB.conll', sep='\t', header=None)
SVM = pd.read_csv('../../data/out_word2vec_added_feats.SVM.conll', sep='\t', header=None)

logreg_out = logreg[4]
logreg_out_actual = logreg[3]
nb_out = NB[4]
nb_out_actual = NB[3]
svm_out = SVM[4]
svm_out_actual = SVM[3]

In [113]:
compare_outcome(logreg_out, logreg_out_actual)
compare_outcome(nb_out, nb_out_actual)
compare_outcome(svm_out, svm_out_actual)

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-PER &     I-LOC &    I-MISC &     I-ORG &     macro \\
\midrule
precision &  0.991264 &  0.938633 &  0.730659 &  0.652690 &  0.618069 &  0.786263 \\
recall    &  0.983813 &  0.731780 &  0.869812 &  0.923852 &  0.813208 &  0.864493 \\
f-score   &  0.987524 &  0.822399 &  0.794186 &  0.764951 &  0.702336 &  0.814279 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
{} &        O &   I-PER &   I-LOC &  I-MISC &  I-ORG \\
\midrule
O      &  41755.0 &   304.0 &     2.0 &      17 &     45 \\
I-PER  &    135.0 &  2952.0 &    22.0 &       5 &     31 \\
I-ORG  &    280.0 &   329.0 &   162.0 &      28 &   1293 \\
I-MISC &    184.0 &   143.0 &    43.0 &     825 &     69 \\
I-LOC  &     88.0 &   306.0 &  1530.0 &      18 &    152 \\
B-MISC &      0.0 &     0.0 &     0.0 &       3 &      1 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrr}
\toprule
{} &         O &     I-PER &     I-LOC &    I-MISC &     I-ORG &     macro \\
